In [8]:
import numpy as np
import pandas as pd
import openpyxl 
import os
%run ccp_func.ipynb

In [15]:
def eccp_colect(quarter):
    
    #quarter = format: 'Q2-2018'
    
    ccp = 'euro-ccp'
    
    #basic
    b_file = pd.read_excel('../CCP/data/euro-ccp-q4-2015-q2-2018.xlsx', sheet_name='AggregatedDataFile')
    #files in use
    a_file = pd.read_excel('../CCP/data/euro-ccp-q4-2015-q2-2018.xlsx', sheet_name='CCP_DataFile_4_4a')
    c_file = pd.read_excel('../CCP/data/euro-ccp-q4-2015-q2-2018.xlsx', sheet_name='CCP_DataFile_6_1')
    d_file = pd.read_excel('../CCP/data/euro-ccp-q4-2015-q2-2018.xlsx', sheet_name='CCP1_DataFile_6.2')
    e_file = pd.read_excel('../CCP/data/euro-ccp-q4-2015-q2-2018.xlsx', sheet_name='CCP_DataFile_18_2')
    g_file = pd.read_excel('../CCP/data/euro-ccp-q4-2015-q2-2018.xlsx', sheet_name='CCP_DataFile_23')
    ex_rate = pd.read_excel('../CCP/data/kursy-walutowe-CCP.xlsx', sheet_name='Arkusz1')
    
    #for 23 calculations there are different files
    h_file = pd.read_excel('../CCP/data/euro_CCP_q1_2017.xlsx', sheet_name='CCP_DataFile_23')
    i_file = pd.read_excel('../CCP/data/euro_CCP_q4_2017.xlsx', sheet_name='CCP_DataFile_23')
    
    if quarter == 'Q2-2018':
        date = '2018-06-30'
    elif quarter == 'Q1-2018':
        date = '2018-03-31'
    elif quarter == 'Q4-2017':
        date = '2017-12-31'
    elif quarter == 'Q3-2017':
        date = '2017-09-30'
    elif quarter == 'Q2-2017':
        date = '2017-06-30'
    elif quarter == 'Q1-2017':
        date = '2017-03-31'
    elif quarter == 'Q4-2016':
        date = '2016-12-31'
    elif quarter == 'Q3-2016':
        date = '2016-09-30'
    elif quarter == 'Q2-2016':
        date = '2016-06-30'
    elif quarter == 'Q1-2016':
        date = '2016-03-31'
    elif quarter == 'Q4-2015':
        date = '2015-12-31'
    elif quarter == 'Q3-2015':
        date = '2015-09-30'
    else:
        pass
    
    #final file
    f_file = pd.DataFrame({'CCP':ccp.capitalize()}, index=[0])
    f_file['quarter'] = quarter
    f_file['clearning service'] = 'aggregated'
    f_file['currency'] = b_file[b_file.ReportDate == date]['Currency'].values
    f_file['4.1.1'] = b_file[b_file.ReportDate == date]['4.1.1'].values
    f_file['4.1.2'] = b_file[b_file.ReportDate == date]['4.1.2'].values
    f_file['4.1.3'] = b_file[b_file.ReportDate == date]['4.1.3'].values
    f_file['4.1.4'] = b_file[b_file.ReportDate == date]['4.1.4'].values
    f_file['4.1.5'] = b_file[b_file.ReportDate == date]['4.1.5'].values
    f_file['4.1.6'] = b_file[b_file.ReportDate == date]['4.1.6'].values
    f_file['4.1.7'] = b_file[b_file.ReportDate == date]['4.1.7'].values
    f_file['4.1.8'] = b_file[b_file.ReportDate == date]['4.1.8'].values
    f_file['4.1.9'] = b_file[b_file.ReportDate == date]['4.1.9'].values
    f_file['4.1.10'] = b_file[b_file.ReportDate == date]['4.1.10'].values
    
    # 4.4.7.PDA & 4.4.7.MA
    f_file['4.4.7.PDA'] = a_file[(a_file.Description == 'PeakDayAmountInPast12Months') & (a_file.ReportDate == date)].reset_index()['4.4.7']
    f_file['4.4.7.MA'] = a_file[(a_file.Description == 'MeanAverageOverPrevious12Months') & (a_file.ReportDate == date)].reset_index()['4.4.7']
    
    #extra col 'ex_rate' in c_file for counting '6.1.1'
    add_ex_rate(c_file, ex_rate, quarter)

    #recalculate c_file to EUR
    c_file['6.1.1'] = c_file['6.1.1'] // c_file['ex_rate']
    
    #6.1.1.CLIENT = 'Client_Net' + 'Client_Gross'
    f_file['6.1.1.CLIENT'] = sum(c_file[c_file.Description.isin(['Client_net', 'Client_Gross']) & (c_file.ReportDate == date)]['6.1.1'])

    #6.1.1.TOTAL = 'House_Net' + 'Client_Net' + 'Client_Gross'
    f_file['6.1.1.TOTAL'] = sum(c_file[c_file.Description.isin(['House_Net', 'Client_net', 'Client_Gross']) & (c_file.ReportDate == date)]['6.1.1'])

    #extra col 'ex_rate' in d_file for counting '6.2.15'
    add_ex_rate(d_file, ex_rate, quarter)
        
    #recalculate d_file to EUR
    d_file['6.2.15'] = d_file['6.2.15'] // d_file['ex_rate']
        
    #6.2.15.PRE & POST
    f_file['6.2.15.PRE'] = sum(d_file[d_file.Description.isin(['HouseIM_PreHaircut', 'ClientIM_PreHaircut']) & (d_file.ReportDate == date)]['6.2.15'])
    f_file['6.2.15.POST'] = sum(d_file[d_file.Description.isin(['HouseIM_PostHaircut', 'ClientIM_PostHaircut']) & (d_file.ReportDate == date)]['6.2.15'])

    #number of CCP
    f_file['18.1.1.1'] = b_file[b_file.ReportDate == date]['18.1.1.1'].values
    f_file['18.1.1.2'] = b_file[b_file.ReportDate == date]['18.1.1.2'].values
    
    #number od CCP participants (average)
    f_file['18.1.2.2'] = b_file[b_file.ReportDate == date]['18.1.2.2'].values
    
    #number of clients
    f_file['19.1.1'] = b_file[b_file.ReportDate == date]['19.1.1'].values
    f_file['19.1.2'] = b_file[b_file.ReportDate == date]['19.1.2'].values
    
    ####
    #extra col 'ex_rate' in g_file for counting '23.1.12' and '23.2.1'
    add_ex_rate(g_file, ex_rate, quarter)
        
    #recalculate g_file to EUR
    g_file['23.1.2'] = g_file['23.1.2'] // g_file['ex_rate']
    g_file['23.2.1'] = g_file['23.2.1'] // g_file['ex_rate']
        
    #23.1.2.OTC & ETD
    f_file['23.1.2.OTC'] = sum(g_file[g_file.Description.isin(['OTC']) & (g_file.ReportDate == date)]['23.1.2'])
    f_file['23.1.2.ETD'] = sum(g_file[g_file.Description.isin(['ETD']) & (g_file.ReportDate == date)]['23.1.2'])
    
    #23.2.1.OTC & ETD
    f_file['23.2.1.OTC'] = sum(g_file[g_file.Description.isin(['OTC']) & (g_file.ReportDate == date)]['23.2.1'])
    f_file['23.2.1.ETD'] = sum(g_file[g_file.Description.isin(['ETD']) & (g_file.ReportDate == date)]['23.2.1'])
    
    writer = pd.ExcelWriter("data/" + ccp.lower() + '/' + ccp.lower() + '-' + quarter.upper() + '.xlsx')
    f_file.to_excel(writer, ccp.lower() + '-' + quarter.upper())
    writer.save()

In [14]:
eccp_colect('Q1-2018')